In [396]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

from functions.latlng2pentad import latlng2pentad
from functions.pentad2latlng import pentad2latlng

A card is defined by three elements: pentad_code, user_id, and date of the first day of the 5 days.

From a card id {pentad}_{observer}_{date}, it is possible to find all checklists that belong to it.

The aim here is to build a list of valid cards which will then be used to find the checklist_id which belongs to this card and then finally compute the card info from the list of checklists.

Pentad: we need to assign for each checklist its pentad and check that the distance traveled is within the boundary of the pentad.
User_id is quite straightforward to build.
Date: much more challenging. See below for details.

## Set up the Import Options and import the data


In [ ]:
cntr = "KE"

In [319]:
ebd0 = pd.read_csv(f"data/eBird/ebd_{cntr}_relAug-2022/ebd_{cntr}_relAug-2022.txt", 
                   delimiter="\t",
                   usecols=["SAMPLING EVENT IDENTIFIER", "SCIENTIFIC NAME", "CATEGORY", "LATITUDE", "LONGITUDE", "OBSERVATION DATE", "TIME OBSERVATIONS STARTED", "PROTOCOL TYPE", "DURATION MINUTES", "EFFORT DISTANCE KM", "ALL SPECIES REPORTED", "OBSERVER ID"],
                   parse_dates=["OBSERVATION DATE"])

In [320]:
# Create OBSERVATIONDATETIME by combining date and time
tmp = ebd0['TIME OBSERVATIONS STARTED'].fillna("00:00:00")
ebd0['OBSERVATION DATETIME'] = pd.to_datetime(ebd0['OBSERVATION DATE'].dt.strftime('%Y-%m-%d') + " " + tmp, format='%Y-%m-%d %H:%M:%S')

# Sort by date: Important to have for filtering duplicate card-adu and needed for sequence
ebd0.sort_values(by="OBSERVATION DATE", inplace=True)

# Keep only species category
# ebd0[["COMMONNAME", "SCIENTIFIC NAME", "CATEGORY"]].drop_duplicates().to_csv("species_list_ebird.csv", index=False)

# Keep some spuh which can be matched to an ADU
# spuh_keep = pd.read_csv("data/spuh_keep.csv", dtype=str)
# ebd0 = ebd0[(~ebd0["CATEGORY"].isin(["spuh", "slash"])) | ebd0["SCIENTIFIC NAME"].isin(spuh_keep["Clements--scientific_name"])]

In [332]:
# Read species_match data
species_match = pd.read_excel("data/World list - working v1.xlsx")
species_match.rename(columns={"Sci (eBird)": "SCIENTIFIC NAME"}, inplace=True)
species_match = species_match[~species_match["ADU"].isna() & ~species_match["SCIENTIFIC NAME"].isna()][["ADU", "SCIENTIFIC NAME"]]

# Check that all entries are matching
matching_entries = ebd0[~ebd0["SCIENTIFIC NAME"].isin(species_match["SCIENTIFIC NAME"])]
matching_entries.groupby("SCIENTIFIC NAME")["SCIENTIFIC NAME"].count().sort_values()

SCIENTIFIC NAME
Phyllastrephus sp.                1
Ardea sp.                         1
Ardeidae sp.                      1
Ardenna grisea/tenuirostris       1
Otididae sp.                      1
                               ... 
Ortygornis sephaena            5450
Eurocephalus ruppelli          5737
Microcarbo africanus           7540
Buphagus erythrorynchus        7908
Crithagra striolata            8122
Name: SCIENTIFIC NAME, Length: 262, dtype: int64

## Build checklist level dataframe


In [323]:
# 
ebd = ebd0.copy()
ebd.drop_duplicates(["SAMPLING EVENT IDENTIFIER", "LATITUDE", "LONGITUDE", "OBSERVATION DATE", "OBSERVATION DATETIME", "PROTOCOL TYPE", "DURATION MINUTES", "EFFORT DISTANCE KM", "ALL SPECIES REPORTED", "OBSERVER ID"], inplace=True)
ebd.sort_values(by="OBSERVATION DATE", inplace=True)

# Filter protocol
ebd["KEEP PROTOCOL"] = ebd["PROTOCOL TYPE"].isin(["Historical", "Incidental", "Stationary", "Traveling"])

# Pentad
# Assign to pentad and check if distance remains inside
ebd["PENTAD"] = latlng2pentad(ebd["LATITUDE"], ebd["LONGITUDE"])

# Search center of pentad
lat, lon = pentad2latlng(ebd["PENTAD"])
dist = (5 / 60 / 2) * 1.2  # allow for a 20% overlap

effort_distance_deg = (180/np.pi) *  ebd["EFFORT DISTANCE KM"] / 6371
ebd["KEEP PENTAD"] = ~((effort_distance_deg + np.maximum(np.abs(lat - ebd["LATITUDE"]), np.abs(lon - ebd["LONGITUDE"]))) > dist)

# Also filter historical checklists which have no distance
ebd.loc[(ebd["PROTOCOL TYPE"] == "Historical") & ebd["EFFORT DISTANCE KM"].isna(), "KEEP PENTAD"] = False

## Find valid full protocol card

Cards are considered to be full protocol if the sum of durations of the underlying checklists exceed 2 hours over the next rolling 5 days.
In this section, we first indentify which checklists can create a valid full card.


In [324]:
# Find the index of all checklists which contribute to the 2hr rule. Note that we will still use "non-valid" checklists later as their species still contribute to the card.
valid_id = (ebd["KEEP PENTAD"] & ebd["KEEP PROTOCOL"] & (ebd["DURATION MINUTES"] > 0) & ebd["ALL SPECIES REPORTED"])

# Filter for valid checklist and create in a smaller table
check = ebd.loc[valid_id, ["PENTAD", "OBSERVER ID", "OBSERVATION DATE", "DURATION MINUTES"]]

# Combine checklists made by the same observer, pentad, and day. This is an intermediate step which enables us to grid the 5days windows more easily
checkday = check.groupby(["PENTAD", "OBSERVER ID", "OBSERVATION DATE"]).agg({"DURATION MINUTES": "sum"}).reset_index()

# Sort the checklist by id and date
checkday.sort_values(by=["OBSERVATION DATE"], inplace=True)

# Create additional columns
checkday["pentad_observer"] = checkday["PENTAD"] + "_" + checkday["OBSERVER ID"]
checkday["pentad_observer_date"] = checkday["PENTAD"] + "_" + checkday["OBSERVER ID"].str[3:] + "_" + checkday["OBSERVATION DATE"].dt.strftime("%Y%m%d")

In [325]:
# Initialize card column. We will only assign a card to checklists contributing to a full protcol card.
checkday["CARD"] = ""

# Loop through each pentad_observer
for po in checkday["pentad_observer"].unique():
    pentad_observer = checkday[checkday["pentad_observer"] == po].index
    di = np.abs(checkday.loc[pentad_observer, "OBSERVATION DATE"].values[:, None] - checkday.loc[pentad_observer, "OBSERVATION DATE"].values) < pd.Timedelta(days=5)
    u = 1
    while u <= len(pentad_observer):
        nb_neighbor = np.sum(di[u-1, (u-1):])
        neigh = u + np.arange(0, nb_neighbor)
        dur = checkday.loc[pentad_observer[neigh-1], "DURATION MINUTES"].sum()
        if dur >= (2*60):
            checkday.loc[pentad_observer[neigh-1], "CARD"] = checkday.loc[pentad_observer[u-1], "pentad_observer_date"]
        u += nb_neighbor

# Create the card DataFrame
card = checkday[checkday["CARD"] == checkday["pentad_observer_date"]][["PENTAD", "OBSERVER ID", "OBSERVATION DATE", "CARD"]]

# Sort by card
card.sort_values(by='CARD', inplace=True)

## Create Card

We take back `ebd` where all checklists (i.e., including the incidentals, stationary, etc...) and find if they contribute to an existing full card.


In [326]:
# Initialize the 'card' column in 'ebd' DataFrame
ebd['CARD'] = ""

# Initialize a list of dictionaries to store the converted data
d = []# [{} for _ in range(len(card))]

for index, row in card.iterrows():
    id = (ebd['KEEP PENTAD'] &
          (ebd['PENTAD'] == row['PENTAD']) &
          (ebd['OBSERVER ID'] == row['OBSERVER ID']) &
          (ebd['OBSERVATION DATE'] >= row['OBSERVATION DATE']) &
          (ebd['OBSERVATION DATE'] < row['OBSERVATION DATE'] + pd.Timedelta(5, unit="D")))

    assert all(ebd.loc[id, 'CARD'] == "")
    ebd.loc[id, 'CARD'] = row['CARD']

    # Create a dictionary for each card
    d.append({
        'Protocol': 'F',
        'ObserverEmail': 'kenyabirdmap@naturekenya.org',
        'CardNo': row['CARD'],
        'StartDate': str(ebd.loc[id, 'OBSERVATION DATE'].min().date()),
        'EndDate': str(ebd.loc[id, 'OBSERVATION DATE'].max().date()),
        'StartTime': str(ebd.loc[id, 'OBSERVATION DATETIME'].min().time()),
        'Pentad': row['PENTAD'],
        'ObserverNo': '22829',
        'TotalHours': np.nansum(ebd.loc[id, 'DURATION MINUTES']) / 60,
        'Hour1': "",
        'Hour2': "",
        'Hour3': "",
        'Hour4': "",
        'Hour5': "",
        'Hour6': "",
        'Hour7': "",
        'Hour8': "",
        'Hour9': "",
        'Hour10': "",
        'TotalSpp': 0,
        'InclNight': "0",
        'AllHabitats': "0",
        'Checklists': ebd.loc[id, 'SAMPLING EVENT IDENTIFIER'].tolist(),
        'TotalDistance': np.nansum(ebd.loc[id, 'EFFORT DISTANCE KM']),
        'ObserverNoEbird': row['OBSERVER ID']
    })

# 'd' now contains the converted data for each card

## Get species level information


In [385]:
# Filter the full dataset to get only the checklist  valid...
all_checklists = [item for x in d for item in x["Checklists"]]
ebd0f = ebd0.loc[ebd0["SAMPLING EVENT IDENTIFIER"].isin(all_checklists),["SAMPLING EVENT IDENTIFIER", "SCIENTIFIC NAME", "OBSERVATION DATETIME"]]

# Add card No
ebd0f = pd.merge(ebd0f, ebd.loc[:,["SAMPLING EVENT IDENTIFIER", "CARD"]], left_on="SAMPLING EVENT IDENTIFIER", right_on="SAMPLING EVENT IDENTIFIER", how="left")

# Keep a unique list of card-species (remove duplicate species in the same card)
ebd0f.sort_values(by="OBSERVATION DATETIME", inplace=True) # SHould have been done already above, but necessary for keep="first"
ebd0fu = ebd0f.drop_duplicates(subset=["CARD", "SCIENTIFIC NAME"], keep='first')


In [386]:
# Add ADU number
ebd0fus = pd.merge(ebd0fu, species_match, how="left")

# Set un-macthed species to undefined
ebd0fus['ADU'] = ebd0fus['ADU'].fillna(0)

In [387]:
# Compute the sequence of records based on datetime entry
ebd0fus["SEQ"] = ebd0fus.groupby("CARD")["OBSERVATION DATETIME"].rank(method="min") # dense t

In [388]:
# Extract the species list per card as a cell for vectorized computation
card_list = ebd0fus.groupby("CARD")[["ADU", "SEQ"]].agg(list).reset_index()

In [389]:
for i, c in card_list.iterrows():
    
    assert(c["CARD"] == d[i]["CardNo"])
    d[i]["records"] = []

    for adu, seq in zip(c["ADU"], c["SEQ"]):
        d[i]["records"].append({
            "Sequence": seq,
            "Latitude": "",
            "Longitude": "",
            "Altitude": "",
            "CardNo": d[i]["CardNo"],
            "Spp": adu,
            "Accuracy": "",
            "SightingTime": ""
        })

    d[i]["TotalSpp"] = len(c["ADU"])

## Export


In [402]:
with open(f"export/{cntr}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json", 'w') as f:
    json.dump(d, f)